In [ ]:
### 入力文とマッチさせる辞書
sentence = 'ピエトロ氏、ドレッシングはピエロ事件。'
dic = {
    'エロ': 'ero',
    'シング': 'sing',
    'ピエトロ': 'pietro',
    '良い天気': 'yoitenki',
    '事件': 'jiken',
}
print('入力文:', sentence)

### （C1）正規表現でマッチさせる
import re
pattern = '|'.join(map(lambda x: re.escape(x),
                   sorted(dic.keys(), key=len, reverse=True)))
for m in re.finditer(pattern, sentence):
    print("MATCH!", m.span(), m.string[m.start():m.end()])

In [ ]:
### テキスト解析 Web API の日本語形態素解析を用いて、テキストを形態素の表層文字列のリストに変換
def sentence_to_tokens(query):
    import json
    from urllib import request
    APPID = "あなたの Client ID（アプリケーション ID）"
    URL = "https://jlp.yahooapis.jp/MAService/V2/parse"
    params = json.dumps({
        "id": "1234-1",
        "jsonrpc": "2.0",
        "method": "jlp.maservice.parse",
        "params": {"q": query}
    }).encode()
    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Yahoo AppID: {}".format(APPID),
    }
    req = request.Request(URL, params, headers)
    with request.urlopen(req) as res:
        body = res.read()
    obj = json.loads(body.decode())
    return [x[0] for x in obj['result']['tokens']]


### 入力文とマッチさせる辞書
sentence = 'ピエトロ氏、ドレッシングはピエロ事件。'
dic = {
    'エロ': 'ero',
    'シング': 'sing',
    'ピエトロ': 'pietro',
    '良い天気': 'yoitenki',
    '事件': 'jiken',
}
print('入力文:', sentence)

### （M1）文を形態素に分割
tokens = sentence_to_tokens(sentence)
print('形態素:', tokens)

### （M2）形態素区切り位置を保存
kugiri_positions = set(sum(len(token) for token in tokens[:i])
                       for i in range(0, len(tokens) + 1))
print('区切り位置:', kugiri_positions)

### （M3）辞書にマッチさせる
for current_position in kugiri_positions:
    for entry in dic.keys():
        end_position = current_position + len(entry)
        if end_position not in kugiri_positions:
            continue
        if sentence[current_position:].startswith(entry):
            print("MATCH!", (current_position, end_position), entry, dic[entry])